![image.png](attachment:image.png)

In [2]:
pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=cd052ed2da3227b910a4dd1d84f24de73d91f00aa25ac798283d227b337d6fa8
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


# NLP Using PySpark

In [14]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, StringIndexer, IDF
from pyspark.ml import Pipeline
from pyspark.ml.classification import NaiveBayes
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.functions import length
from pyspark.ml.feature import VectorAssembler


##Spam filter using NaiveBayes classifier:


In [4]:
#Spark session
spark = SparkSession.builder \
    .appName("SpamFilter") \
    .getOrCreate()

In [6]:
data = spark.read.option("sep", "\t").csv("/content/SMSSpamCollection", inferSchema=True, header=False)

In [7]:
data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- _c1: string (nullable = true)



In [8]:
data = data.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")

In [9]:
data.show(10, truncate=True)
data.show(10, truncate=False)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
|  ham|U dun say so earl...|
|  ham|Nah I don't think...|
| spam|FreeMsg Hey there...|
|  ham|Even my brother i...|
|  ham|As per your reque...|
| spam|WINNER!! As a val...|
| spam|Had your mobile 1...|
+-----+--------------------+
only showing top 10 rows

+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|class|text                                                                                                                                                            |
+-----+----------------------------------------------------------------------------------------------------------------------------------------------------------------+
|ham  |Go until jurong point, crazy.. Available only in bugis

## Data Preparation and Cleaning

In [10]:
data = data.withColumn("length", length("text"))
data.show()

+-----+--------------------+------+
|class|                text|length|
+-----+--------------------+------+
|  ham|Go until jurong p...|   111|
|  ham|Ok lar... Joking ...|    29|
| spam|Free entry in 2 a...|   155|
|  ham|U dun say so earl...|    49|
|  ham|Nah I don't think...|    61|
| spam|FreeMsg Hey there...|   147|
|  ham|Even my brother i...|    77|
|  ham|As per your reque...|   160|
| spam|WINNER!! As a val...|   157|
| spam|Had your mobile 1...|   154|
|  ham|I'm gonna be home...|   109|
| spam|SIX chances to wi...|   136|
| spam|URGENT! You have ...|   155|
|  ham|I've been searchi...|   196|
|  ham|I HAVE A DATE ON ...|    35|
| spam|XXXMobileMovieClu...|   149|
|  ham|Oh k...i'm watchi...|    26|
|  ham|Eh u remember how...|    81|
|  ham|Fine if thats th...|    56|
| spam|England v Macedon...|   155|
+-----+--------------------+------+
only showing top 20 rows



In [11]:
avg_length = data.groupBy("class").avg("length").withColumnRenamed("avg(length)", "Avg_Length")
avg_length.show()

+-----+-----------------+
|class|       Avg_Length|
+-----+-----------------+
|  ham|71.45431945307645|
| spam|138.6706827309237|
+-----+-----------------+



## Feature Transformations

In [12]:
tokenizer = Tokenizer(inputCol="text", outputCol="token_text")
stop_words_remover = StopWordsRemover(inputCol="token_text", outputCol="stop_tokens")
count_vec = CountVectorizer(inputCol="stop_tokens", outputCol="c_vec")
idf = IDF(inputCol="c_vec", outputCol="tf_idf")

In [16]:
indexer = StringIndexer(inputCol="class", outputCol="label")
assembler = VectorAssembler(inputCols=["tf_idf", "length"], outputCol="features")

## The Model


In [21]:
# Create a NaiveBayes classifier
nb = NaiveBayes()


## Pipeline


In [23]:
# a pipeline model
pipeline = Pipeline(stages=[tokenizer, stop_words_remover, count_vec, idf, indexer, assembler, nb])

In [24]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [25]:
model = pipeline.fit(train_data)

In [26]:
predictions = model.transform(test_data)

### Print the schema of the prediction dataframe

In [21]:
predictions.printSchema()

root
 |-- class: string (nullable = true)
 |-- text: string (nullable = true)
 |-- length: integer (nullable = true)
 |-- token_text: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- stop_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- c_vec: vector (nullable = true)
 |-- tf_idf: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



## Model Evaluation

In [27]:
evaluator = MulticlassClassificationEvaluator(predictionCol="prediction", labelCol="label", metricName="f1")
f1_score = evaluator.evaluate(predictions)
print("f1_score is:", f1_score)

f1_score is: 0.9826402170280536


In [28]:

#Predicting with real SMS after evalution
custom_sms = spark.createDataFrame([(0, "Hello son dinner is ready")], ["id", "text"])
custom_sms = custom_sms.withColumn("length", length("text"))
custom_predictions = model.transform(custom_sms)
custom_predictions.select("text", "prediction").show()

+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|Hello son dinner ...|       0.0|
+--------------------+----------+



In [29]:
custom_sms = spark.createDataFrame([(0, "Hello son dinner is ready"),
                                    (1, "free access send visa"),
                                    (2, "meeting is cancelled")],
                                   ["id", "text"])
custom_sms = custom_sms.withColumn("length", length("text"))
custom_predictions = model.transform(custom_sms)
custom_predictions.select("text", "prediction").show()

+--------------------+----------+
|                text|prediction|
+--------------------+----------+
|Hello son dinner ...|       0.0|
|free access send ...|       1.0|
|meeting is cancelled|       0.0|
+--------------------+----------+



In [26]:
model.save("path_to_save_model")

In [28]:
!mv path_to_save_model "/content/drive/My Drive/"


mv: cannot move 'path_to_save_model' to '/content/drive/My Drive/': No such file or directory


In [29]:
pwd

'/content'

In [30]:
import shutil
colab_link = "/content/path_to_save_model"
gdrive_link = "/content/drive/MyDrive/"
shutil.copy(colab_link, gdrive_link)

IsADirectoryError: [Errno 21] Is a directory: '/content/path_to_save_model'

In [31]:
import shutil

colab_directory = "/content/path_to_save_model"
gdrive_destination = "/content/drive/MyDrive/"

shutil.copytree(colab_directory, gdrive_destination)


'/content/drive/MyDrive/'

In [32]:

colab_directory = "/content/path_to_save_model"
gdrive_destination = "/content/drive/MyDrive/"

shutil.copytree(colab_directory, gdrive_destination)

FileExistsError: [Errno 17] File exists: '/content/drive/MyDrive/'

In [33]:
from google.colab import files
files.download("/content/path_to_save_model")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# GOOD LUCK

![image-3.png](attachment:image-3.png)